# Create a new data set only with the images that have a TN 

### imports

In [101]:
import pandas as pd
from pathlib import Path
import shutil
import os

In [102]:
dataset_root    = Path('dataset/Dataset for Fetus Framework')
excel_path      = Path('ObjectDetection.xlsx')
new_dataset_root= Path('new_dataset')
train_val_dir   = new_dataset_root / 'train_val'
test_dir        = new_dataset_root / 'test'

Start by analizing on the actual data set how many standart images have a tn 

In [103]:
THIS_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(THIS_DIR, os.pardir))
DATASET_ROOT      = os.path.join(PROJECT_ROOT, 'dataset', 'Dataset for Fetus Framework')
EXCEL_PATH        = os.path.join(PROJECT_ROOT,'dataset', 'ObjectDetection.xlsx')

# 4) Where to dump the reorganized dataset
NEW_ROOT          = os.path.join(PROJECT_ROOT, 'pre_processing_segmentation')
NEW_TRAIN_VAL_DIR = os.path.join(NEW_ROOT, 'train_val')
NEW_TEST_DIR      = os.path.join(NEW_ROOT, 'test')

# ─────────────────────────────────────────────────────

def gather_images(dir_list):
    imgs = []
    for d in dir_list:
        for fname in os.listdir(d):
            f = os.path.join(d, fname)
            if os.path.isfile(f):
                imgs.append(f)
    return imgs

# ─────────────────────────────────────────────────────────────



In [104]:
df = pd.read_excel(EXCEL_PATH)

In [105]:
PROJECT_ROOT

'c:\\Users\\ibajl\\Desktop\\iamedic-demo'

In [106]:
print(DATASET_ROOT)

c:\Users\ibajl\Desktop\iamedic-demo\dataset\Dataset for Fetus Framework


In [107]:

train_val_dirs = [
    os.path.join(DATASET_ROOT, 'Set1-Training&Validation Sets CNN',         'Standard'),
    os.path.join(DATASET_ROOT, 'Set2-Training&Validation Sets ANN Scoring system', 'Standard'),
]
test_dirs = [
    os.path.join(DATASET_ROOT, 'External Test Set', 'Standard'),
    os.path.join(DATASET_ROOT, 'Internal Test Set', 'Standard'),
]

In [108]:
# 3) Collect all image files
train_val_imgs = gather_images(train_val_dirs)
test_imgs      = gather_images(test_dirs)
all_standard   = train_val_imgs + test_imgs

In [109]:
len(train_val_imgs), len(test_imgs)

(1061, 136)

In [110]:
def count_files(path):
    return len([
        name for name in os.listdir(path)
        if os.path.isfile(os.path.join(path, name))
    ])

In [111]:
train_file = count_files( train_val_dirs[0]) + count_files(train_val_dirs[1])
test_files = count_files(test_dirs[0]) + count_files(test_dirs[1])
print(f'Train files: {train_file}, Test files: {test_files}')

Train files: 1061, Test files: 136


In [112]:
df

,fname,structure,h_min,w_min,h_max,w_max
0,168.png,thalami,178,171,244,261
1,168.png,nasal bone,96,308,111,349
2,168.png,palate,133,300,205,408
3,168.png,nasal skin,86,324,95,349
4,168.png,nasal tip,79,345,89,376
...,...,...,...,...,...,...
9428,1577.png,nasal tip,134,363,142,385
9429,1577.png,NT,354,240,374,392
9430,1577.png,palate,169,317,219,407
9431,1577.png,thalami,192,217,244,281


In [113]:
# 1) Build the set of *all* NT‐labeled filenames from your Excel
tn_basenames = {
    os.path.basename(p)
    for p in df[df['structure'].str.upper() == 'NT']['fname']
}
print(f"Total NT images in Excel: {len(tn_basenames)}")

# 2) Now intersect with your standard image lists
standard_basenames   = {os.path.basename(p) for p in all_standard}
train_val_basenames  = {os.path.basename(p) for p in train_val_imgs}
test_basenames       = {os.path.basename(p) for p in test_imgs}

# a) Of those NTs, how many are even in your Standard subset?
standard_tn          = standard_basenames  & tn_basenames
print(f"  → Standard NT images overall: {len(standard_tn)}")

# b) How many of those end up in Train/Val?
train_val_tn         = train_val_basenames & tn_basenames
print(f"  →  Train/Val NT images:        {len(train_val_tn)}")

# c) How many end up in Test?
test_tn              = test_basenames      & tn_basenames
print(f"  →  Test NT images:             {len(test_tn)}")


Total NT images in Excel: 1110
  → Standard NT images overall: 1110
  →  Train/Val NT images:        1038
  →  Test NT images:             72


In [114]:
images_no_NT_train_val = train_val_basenames - train_val_tn
images_no_NT_test      = test_basenames      - test_tn
print(f"  →  Train/Val images without NT: {len(images_no_NT_train_val)}")
print(f"  →  Test images without NT:        {len(images_no_NT_test)}")

  →  Train/Val images without NT: 23
  →  Test images without NT:        64


In [115]:
# 5) Make the output folders
os.makedirs(NEW_TRAIN_VAL_DIR, exist_ok=True)
os.makedirs(NEW_TEST_DIR,      exist_ok=True)



In [116]:
# train/val NT images
for src in train_val_imgs:
    if os.path.basename(src) in tn_basenames:
        shutil.copy(src,
                    os.path.join(NEW_TRAIN_VAL_DIR,
                                 os.path.basename(src)))

# test NT images
for src in test_imgs:
    if os.path.basename(src) in tn_basenames:
        shutil.copy(src,
                    os.path.join(NEW_TEST_DIR,
                                 os.path.basename(src)))

In [117]:
train_val_tn = train_val_basenames & tn_basenames
test_tn      = test_basenames      & tn_basenames

In [118]:
# 6a) Copy train/val NT images
for src in train_val_imgs:
    name = os.path.basename(src)
    if name in train_val_tn:
        shutil.copy(src,
                    os.path.join(NEW_TRAIN_VAL_DIR, name))

# 6b) Copy test NT images
for src in test_imgs:
    name = os.path.basename(src)
    if name in test_tn:
        shutil.copy(src,
                    os.path.join(NEW_TEST_DIR, name))

In [119]:
prosses_test  = count_files(NEW_TEST_DIR)
prosses_train = count_files(NEW_TRAIN_VAL_DIR)
print(f'Processed test files: {prosses_test}, Processed train files: {prosses_train}')

Processed test files: 72, Processed train files: 1038


In [125]:
# Reload the original Excel file to ensure df is not empty
df = pd.read_excel(EXCEL_PATH)

# 1) Build the set of Standard basenames
standard_basenames = {
    os.path.basename(p)
    for p in all_standard
}

# 2) Within your DataFrame, find which of those have at least one TN
df['basename'] = df['fname'].map(lambda p: os.path.basename(p))
mask_standard = df['basename'].isin(standard_basenames)
mask_tn       = df['structure'].str.strip().str.upper() == 'NT'

# Rows that are both Standard and TN
standard_tn_rows = df[mask_standard & mask_tn]

# The set of image names to keep
standard_tn_basenames = set(standard_tn_rows['basename'])
print(f"Found {len(standard_tn_basenames)} Standard images labeled TN.")

# 3) Now select ALL rows whose basename is in that TN set AND is Standard
mask_export = df['basename'].isin(standard_tn_basenames) & mask_standard
df_export  = df[mask_export].copy()

print(f"Exporting {len(df_export)} rows (all structures for those images).")

# 4) Write out only those rows
out_csv = os.path.join(NEW_ROOT, 'standard_labels.csv')
df_export.to_csv(out_csv, index=False)
print("Written:", out_csv)

Found 1110 Standard images labeled TN.
Exporting 9287 rows (all structures for those images).
Written: c:\Users\ibajl\Desktop\iamedic-demo\pre_processing_segmentation\standard_labels.csv


In [126]:
# Point at your CSV
csv_path = r"C:\Users\ibajl\Desktop\iamedic-demo\pre_processing_segmentation\standard_labels.csv"

# Load and count
df = pd.read_csv(csv_path)
tn_count = (df['structure'].str.upper() == 'NT').sum()

print(f"Number of rows labeled TN: {tn_count}")

Number of rows labeled TN: 1110


In [127]:
df2 = pd.read_csv(csv_path)

In [128]:
df2

,fname,structure,h_min,w_min,h_max,w_max,basename
0,168.png,thalami,178,171,244,261,168.png
1,168.png,nasal bone,96,308,111,349,168.png
2,168.png,palate,133,300,205,408,168.png
3,168.png,nasal skin,86,324,95,349,168.png
4,168.png,nasal tip,79,345,89,376,168.png
...,...,...,...,...,...,...,...
9282,1577.png,nasal tip,134,363,142,385,1577.png
9283,1577.png,NT,354,240,374,392,1577.png
9284,1577.png,palate,169,317,219,407,1577.png
9285,1577.png,thalami,192,217,244,281,1577.png
